## Preparing Data


In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.read_excel('data/data.xlsx')

In [49]:
df.head().T

,0,1,2,3,4
yyyy,1947.000000,1948.000000,1949.000000,1950.000000,1951.000000
Index,15.300000,15.200000,16.760000,20.410000,23.770000
D12,0.840000,0.930000,1.140000,1.470000,1.410000
E12,1.610000,2.290000,2.320000,2.840000,2.440000
b/m,0.725326,0.840948,0.796429,0.722538,0.721316
tbl,0.009500,0.011600,0.011000,0.013400,0.017300
AAA,0.028600,0.027900,0.025800,0.026700,0.030100
BAA,0.035200,0.035300,0.033100,0.032000,0.036100
lty,0.024300,0.023700,0.020900,0.022400,0.026900
cay,0.079056,0.054523,0.051693,0.004356,0.004147


In [50]:
# Our Target Equity Premium
df["erp"] = df["CRSP_SPvw"] - df["Rfree"]

# Data preparing
df['lag_idx'] = df['Index'].shift(1)


# Features
df['dp'] = np.log(df['D12']/df['Index'])
df["dy"] = np.log(df["D12"] / (df["lag_idx"]))
df["ep"] = np.log(df["E12"] / df["Index"])
df['de'] = np.log(df['D12']/df['E12'])
df['tms'] = df['lty'] - df['tbl']
df['dfy'] = df['BAA'] - df['AAA']
df['dfr'] = df['lty'] - df['corpr']

# Cleaning data
df = df.dropna()

## Problem
The full sample period is 1947-2024. The initial estimation window is 1947-1966. The out-of-sample
period covers 1967-2024. The whole raw data from Goyal and Welch is in the attached EXCEL file
(PredictorData2024.xlsx).

(a) Use the historical mean model to calculate the 1-step forecasts, forecast errors and MSE for the out-of-sample period.

(b) Use AR(1) model to calculate the 1-step forecasts, forecast errors and MSE for the out-of-sample period.

(c) Use each single predictor in the list above to predict the equity premium, calculate the 1-step forecasts, forecast errors and MSEs for the out-of-sample period. (hint: you need to do 16 predictive regression each with a single predictor in the right hand side of the regression)

(d) Use all the predictors in one multivariate regression to predict the equity premium, calculate the 1-step forecasts, forecast errors and MSEs for the out-of-sample period.

(e) Equally weight the 16 forecasts from (c) to calculate the combined forecast. Calculate the forecast errors and MSE from this combined forecast for the out-of-sample period.

(f) Compare the out-of-sample MSE of the forecasts in (a), (b), (c) , (d), and (e), and discuss which model is the best if we use the out-of-sample MSE to evaluate the predictive performance for the out-of-sample period.

(g) Use the best model chosen in (f) to forecast the annual equity premium in 2025

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

oos_years = range(1967, 2025)
actuals, forecasts= [], []

## (a) Historical Mean Model

In [52]:
actuals, forecasts = [], []
for t in oos_years:
    train_mask = df["yyyy"] < t
    test_mask = df["yyyy"] == t

    mean = df.loc[train_mask, "erp"].mean()
    
    forecasts.append(mean)
    actual = df.loc[test_mask, "erp"].values[0]
    actuals.append(actual)

mse = mean_squared_error(actuals, forecasts)
print(f"Mean Squared Error: {mse}")
print(f"Number of forecasts: {len(forecasts)}")

# Display year-by-year results
results_by_year = pd.DataFrame({
    'Year': list(oos_years),
    'Actual': actuals,
    'Forecast': forecasts,
    'Forecast Error': np.array(actuals) - np.array(forecasts),
    'Squared Error': (np.array(actuals) - np.array(forecasts))**2
})

print("\n=== Year-by-Year Results (Part a) ===")
print(results_by_year.to_string(index=False))

Mean Squared Error: 0.0305578730409191
Number of forecasts: 58

=== Year-by-Year Results (Part a) ===
 Year    Actual  Forecast  Forecast Error  Squared Error
 1967  0.195972  0.130323        0.065649       0.004310
 1968  0.055114  0.133605       -0.078491       0.006161
 1969 -0.148501  0.129867       -0.278368       0.077489
 1970 -0.030010  0.117214       -0.147224       0.021675
 1971  0.099550  0.110813       -0.011263       0.000127
 1972  0.149770  0.110344        0.039426       0.001554
 1973 -0.216229  0.111921       -0.328150       0.107683
 1974 -0.345023  0.099300       -0.444323       0.197423
 1975  0.307222  0.082843        0.224379       0.050346
 1976  0.185800  0.090857        0.094943       0.009014
 1977 -0.124637  0.094131       -0.218768       0.047859
 1978 -0.004950  0.086839       -0.091789       0.008425
 1979  0.087764  0.083878        0.003887       0.000015
 1980  0.209821  0.083999        0.125822       0.015831
 1981 -0.202487  0.087812       -0.290299  

In [53]:
df["erp_lag1"] = df["erp"].shift(1)
df[["erp", "erp_lag1"]].head()

,erp,erp_lag1
1,0.042443,NaN
2,0.169100,0.042443
3,0.317732,0.169100
4,0.219621,0.317732
5,0.172491,0.219621


## (b) AR(1) Model


In [61]:
# AR(1) Model
forecasts_ar1 = []
actuals_ar1 = []

for t in oos_years:
    train_mask = df["yyyy"] < t
    test_mask = df["yyyy"] == t

    # Prepare lagged erp
    df["erp_lag1"] = df["erp"].shift(1)

    X_train = df.loc[train_mask, ["erp_lag1"]].dropna()
    y_train = df.loc[train_mask, "erp"].loc[X_train.index]

    model = LinearRegression()
    model.fit(X_train, y_train)

    X_test = df.loc[test_mask, ["erp_lag1"]]
    pred = model.predict(X_test)[0]

    forecasts_ar1.append(pred)
    actuals_ar1.append(df.loc[test_mask, "erp"].values[0])

mse_ar1 = mean_squared_error(actuals_ar1, forecasts_ar1)
print(f"AR(1) MSE: {mse_ar1}")

# Display year-by-year results
results_ar1 = pd.DataFrame({
    'Year': list(oos_years),
    'Actual': actuals_ar1,
    'Forecast': forecasts_ar1,
    'Forecast Error': np.array(actuals_ar1) - np.array(forecasts_ar1)
})

print("\n=== Year-by-Year Forecast Errors (Part b) ===")
print(results_ar1.to_string(index=False))

AR(1) MSE: 0.032175355149323036

=== Year-by-Year Forecast Errors (Part b) ===
 Year    Actual  Forecast  Forecast Error
 1967  0.195972  0.196461       -0.000490
 1968  0.055114  0.124822       -0.069708
 1969 -0.148501  0.151033       -0.299534
 1970 -0.030010  0.170262       -0.200271
 1971  0.099550  0.128782       -0.029231
 1972  0.149770  0.114368        0.035402
 1973 -0.216229  0.111043       -0.327272
 1974 -0.345023  0.138665       -0.483687
 1975  0.307222  0.051767        0.255455
 1976  0.185800  0.084812        0.100988
 1977 -0.124637  0.094623       -0.219259
 1978 -0.004950  0.095615       -0.100566
 1979  0.087764  0.086544        0.001221
 1980  0.209821  0.085285        0.124536
 1981 -0.202487  0.087509       -0.289996
 1982  0.107049  0.093405        0.013645
 1983  0.134852  0.079841        0.055011
 1984 -0.033576  0.080145       -0.113721
 1985  0.240998  0.085435        0.155562
 1986  0.116470  0.073139        0.043331
 1987 -0.007688  0.082664       -0.0903

## (c) Single Predictor

In [55]:
predictors = [
    "dp",
    "dy",
    "ep",
    "de",
    "svar",
    "b/m",
    "ntis",
    "eqis",
    "tbl",
    "lty",
    "ltr",
    "tms",
    "dfy",
    "dfr",
    "infl",
    "ik",
]

# Store results for each predictor
mse_single = {}
forecasts_single = {pred: [] for pred in predictors}
actuals_single = []

for predictor in predictors:
    forecasts_temp = []
    actuals_temp = []

    for t in oos_years:
        train_mask = df["yyyy"] < t
        test_mask = df["yyyy"] == t

        X_train = df.loc[train_mask, [predictor]]
        y_train = df.loc[train_mask, "erp"]

        model = LinearRegression()
        model.fit(X_train, y_train)

        X_test = df.loc[test_mask, [predictor]]
        pred = model.predict(X_test)[0]

        forecasts_temp.append(pred)
        actuals_temp.append(df.loc[test_mask, "erp"].values[0])

    forecasts_single[predictor] = forecasts_temp
    actuals_single = actuals_temp  # Same for all
    mse_single[predictor] = mean_squared_error(actuals_temp, forecasts_temp)

print("Single Predictor MSEs:")
for pred, mse in mse_single.items():
    print(f"{pred}: {mse:.6f}")

# Display year-by-year forecast errors for each predictor
for predictor in predictors:
    results_single = pd.DataFrame({
        'Year': list(oos_years),
        'Actual': actuals_single,
        'Forecast': forecasts_single[predictor],
        'Forecast Error': np.array(actuals_single) - np.array(forecasts_single[predictor])
    })
    print(f"\n=== Year-by-Year Forecast Errors: {predictor} (Part c) ===")
    print(results_single.to_string(index=False))

Single Predictor MSEs:
dp: 0.030999
dy: 0.032529
ep: 0.032458
de: 0.032439
svar: 0.030876
b/m: 0.030820
ntis: 0.032466
eqis: 0.032592
tbl: 0.034504
lty: 0.032500
ltr: 0.033475
tms: 0.034815
dfy: 0.028410
dfr: 0.031010
infl: 0.029231
ik: 0.030279

=== Year-by-Year Forecast Errors: dp (Part c) ===
 Year    Actual  Forecast  Forecast Error
 1967  0.195972  0.120289        0.075683
 1968  0.055114  0.126833       -0.071719
 1969 -0.148501  0.125141       -0.273641
 1970 -0.030010  0.109695       -0.139705
 1971  0.099550  0.094611        0.004939
 1972  0.149770  0.087714        0.062056
 1973 -0.216229  0.107023       -0.323253
 1974 -0.345023  0.120019       -0.465041
 1975  0.307222  0.082956        0.224266
 1976  0.185800  0.090688        0.095111
 1977 -0.124637  0.095698       -0.220335
 1978 -0.004950  0.082179       -0.087130
 1979  0.087764  0.076477        0.011288
 1980  0.209821  0.080665        0.129156
 1981 -0.202487  0.082332       -0.284819
 1982  0.107049  0.069465      

## (d) All Predictor (result from single predictor)

In [56]:
# Multivariate Regression with all predictors
forecasts_multi = []
actuals_multi = []

x_cols = [
    "dp",
    "dy",
    "ep",
    "de",
    "svar",
    "b/m",
    "ntis",
    "eqis",
    "tbl",
    "lty",
    "ltr",
    "tms",
    "dfy",
    "dfr",
    "infl",
    "ik",
]

for t in oos_years:
    train_mask = df["yyyy"] < t
    test_mask = df["yyyy"] == t

    X_train = df.loc[train_mask, x_cols]
    y_train = df.loc[train_mask, "erp"]

    model = LinearRegression()
    model.fit(X_train, y_train)

    X_test = df.loc[test_mask, x_cols]
    pred = model.predict(X_test)[0]

    forecasts_multi.append(pred)
    actuals_multi.append(df.loc[test_mask, "erp"].values[0])

mse_multi = mean_squared_error(actuals_multi, forecasts_multi)
print(f"Multivariate MSE: {mse_multi}")

# Display year-by-year results
results_multi = pd.DataFrame({
    'Year': list(oos_years),
    'Actual': actuals_multi,
    'Forecast': forecasts_multi,
    'Forecast Error': np.array(actuals_multi) - np.array(forecasts_multi)
})

print("\n=== Year-by-Year Forecast Errors (Part d) ===")
print(results_multi.to_string(index=False))

Multivariate MSE: 0.0013415399709559197

=== Year-by-Year Forecast Errors (Part d) ===
 Year    Actual  Forecast  Forecast Error
 1967  0.195972  0.239858       -0.043886
 1968  0.055114  0.057396       -0.002282
 1969 -0.148501 -0.195432        0.046931
 1970 -0.030010  0.033324       -0.063333
 1971  0.099550  0.101384       -0.001833
 1972  0.149770  0.133580        0.016190
 1973 -0.216229 -0.284185        0.067955
 1974 -0.345023 -0.333257       -0.011766
 1975  0.307222  0.310257       -0.003035
 1976  0.185800  0.150094        0.035706
 1977 -0.124637 -0.182329        0.057692
 1978 -0.004950  0.042817       -0.047767
 1979  0.087764  0.114304       -0.026540
 1980  0.209821  0.201062        0.008759
 1981 -0.202487 -0.198090       -0.004397
 1982  0.107049  0.065206        0.041844
 1983  0.134852  0.123694        0.011158
 1984 -0.033576 -0.075581        0.042005
 1985  0.240998  0.201068        0.039930
 1986  0.116470  0.143209       -0.026738
 1987 -0.007688  0.120800      

## (e) Equally Weighted forecast

In [57]:
# Combined forecast - equal weight of 16 single predictor forecasts
forecasts_combined = []

for i in range(len(oos_years)):
    # Average across all 16 predictor forecasts for year i
    avg_forecast = np.mean([forecasts_single[pred][i] for pred in predictors])
    forecasts_combined.append(avg_forecast)

mse_combined = mean_squared_error(actuals_single, forecasts_combined)
print(f"Combined Forecast MSE: {mse_combined}")

# Display year-by-year results
results_combined = pd.DataFrame({
    'Year': list(oos_years),
    'Actual': actuals_single,
    'Forecast': forecasts_combined,
    'Forecast Error': np.array(actuals_single) - np.array(forecasts_combined)
})

print("\n=== Year-by-Year Forecast Errors (Part e) ===")
print(results_combined.to_string(index=False))

Combined Forecast MSE: 0.027541592207370545

=== Year-by-Year Forecast Errors (Part e) ===
 Year    Actual  Forecast  Forecast Error
 1967  0.195972  0.107807        0.088165
 1968  0.055114  0.103623       -0.048509
 1969 -0.148501  0.092442       -0.240943
 1970 -0.030010  0.069411       -0.099420
 1971  0.099550  0.085415        0.014136
 1972  0.149770  0.083816        0.065954
 1973 -0.216229  0.058039       -0.274268
 1974 -0.345023 -0.004730       -0.340293
 1975  0.307222  0.029739        0.277483
 1976  0.185800  0.082716        0.103083
 1977 -0.124637  0.067970       -0.192606
 1978 -0.004950  0.010690       -0.015640
 1979  0.087764 -0.023852        0.111616
 1980  0.209821 -0.014173        0.223994
 1981 -0.202487  0.036452       -0.238939
 1982  0.107049  0.048155        0.058895
 1983  0.134852  0.065869        0.068983
 1984 -0.033576  0.050233       -0.083809
 1985  0.240998  0.069919        0.171079
 1986  0.116470  0.089406        0.027065
 1987 -0.007688  0.043618  

## (f) Compare & Discuss


In [58]:
# Summary of all MSEs
results = {
    "Historical Mean": mse,  # from part (a)
    "AR(1)": mse_ar1,
    "Multivariate": mse_multi,
    "Combined": mse_combined,
}

# Add individual predictor MSEs
for pred, mse_val in mse_single.items():
    results[f"Single: {pred}"] = mse_val

# Sort by MSE
results_df = pd.DataFrame.from_dict(results, orient="index", columns=["MSE"])
results_df = results_df.sort_values("MSE")
print("\n=== MODEL COMPARISON ===")
print(results_df)

best_model = results_df.index[0]
print(f"\nBest Model: {best_model} with MSE = {results_df.iloc[0, 0]:.6f}")


=== MODEL COMPARISON ===
                      MSE
Multivariate     0.001342
Combined         0.027542
Single: dfy      0.028410
Single: infl     0.029231
Historical Mean  0.030279
Single: ik       0.030279
Single: b/m      0.030820
Single: svar     0.030876
Single: dp       0.030999
Single: dfr      0.031010
AR(1)            0.032175
Single: de       0.032439
Single: ep       0.032458
Single: ntis     0.032466
Single: lty      0.032500
Single: dy       0.032529
Single: eqis     0.032592
Single: ltr      0.033475
Single: tbl      0.034504
Single: tms      0.034815

Best Model: Multivariate with MSE = 0.001342


## (g) Best Model

In [59]:
# Forecast for 2025 using the best model
# Train on ALL available data (1947-2024)

if "Historical Mean" in best_model:
    forecast_2025 = df["erp"].mean()
elif "AR(1)" in best_model:
    df["erp_lag1"] = df["erp"].shift(1)
    X_all = df[["erp_lag1"]].dropna()
    y_all = df["erp"].loc[X_all.index]
    model = LinearRegression()
    model.fit(X_all, y_all)
    last_erp = df["erp"].iloc[-1]
    forecast_2025 = model.predict([[last_erp]])[0]
elif "Single:" in best_model:
    pred_name = best_model.split(": ")[1]
    X_all = df[[pred_name]]
    y_all = df["erp"]
    model = LinearRegression()
    model.fit(X_all, y_all)
    # Need 2025 predictor value - use last available or extrapolate
    X_2025 = df[[pred_name]].iloc[-1:]
    forecast_2025 = model.predict(X_2025)[0]
elif "Multivariate" in best_model:
    X_all = df[x_cols]
    y_all = df["erp"]
    model = LinearRegression()
    model.fit(X_all, y_all)
    X_2025 = df[x_cols].iloc[-1:]
    forecast_2025 = model.predict(X_2025)[0]
elif "Combined" in best_model:
    # Average of all 16 single predictor forecasts for 2025
    forecasts_2025 = []
    for pred in predictors:
        X_all = df[[pred]]
        y_all = df["erp"]
        model = LinearRegression()
        model.fit(X_all, y_all)
        X_2025 = df[[pred]].iloc[-1:]
        forecasts_2025.append(model.predict(X_2025)[0])
    forecast_2025 = np.mean(forecasts_2025)

print(f"\n2025 Equity Premium Forecast: {forecast_2025:.4f}")


2025 Equity Premium Forecast: 0.2102
